# TensorFlow Assignment: Multi-Layer Perceptron (MLP)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: Rachel Kositsky

### Multi-layer Perceptron

Build a 2-layer MLP for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image (784 dimensions) -> fully connected layer (500 hidden units)  -> nonlinearity (ReLU) -> fully connected layer (100 hidden units) -> nonlinearity (ReLU) -> fully connected (10 hidden units) -> softmax

In [3]:
### YOUR CODE HERE
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import trange

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Make sure to print out your accuracy on the test set at the end. => RK: got 97% accuracy after 50 rounds of training

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [12]:
# Create MLP
tf.reset_default_graph()
g = tf.get_default_graph()

# Add placeholders for data and labels. Something * 784 pixels.
# Later: X = 100 images, etc.
X = tf.placeholder(tf.float32, [None, 784]) # data
y = tf.placeholder(tf.float32, [None, 10])  # labels

# Add variables for first layer
num_hidden_units_1 = 500
W1 = tf.Variable(tf.truncated_normal([784, num_hidden_units_1], stddev=0.1))  # weight
b1 = tf.Variable(tf.truncated_normal([num_hidden_units_1], stddev=0.1)) # bias
mult_result1 = tf.matmul(X, W1)
latent_scores_1 = tf.add(mult_result1, b1)
latent_relu_1 = tf.nn.relu(latent_scores_1)

# Add variables for second layer
num_hidden_units_2 = 256
W2 = tf.Variable(tf.truncated_normal([num_hidden_units_1, num_hidden_units_2], stddev=0.1))
b2 = tf.Variable(tf.truncated_normal([num_hidden_units_2], stddev=0.1))
mult_result2 = tf.matmul(latent_relu_1, W2)
latent_scores_2 = tf.add(mult_result2, b2)
latent_relu_2 = tf.nn.relu(latent_scores_2)

# Add third layer with scores
num_scores = 10
W3 = tf.Variable(tf.truncated_normal([num_hidden_units_2, num_scores], stddev=0.1))
b3 = tf.Variable(tf.truncated_normal([num_scores], stddev=0.1))
mult_result3 = tf.matmul(latent_relu_2, W3)
scores = tf.add(mult_result3, b3)

# Add softmax loss calculation
loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=scores, labels=y)
avg_loss = tf.reduce_mean(loss)
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(avg_loss)
# learning rate = 0.01. pass it the node that we want it to minimize: the average loss node/the cross entropy

# Initialization
# To implement stochastic gradient descent, will feed in minibatches for X, y
initialize_all = tf.global_variables_initializer()

g.get_operations()

[<tf.Operation 'Placeholder' type=Placeholder>,
 <tf.Operation 'Placeholder_1' type=Placeholder>,
 <tf.Operation 'truncated_normal/shape' type=Const>,
 <tf.Operation 'truncated_normal/mean' type=Const>,
 <tf.Operation 'truncated_normal/stddev' type=Const>,
 <tf.Operation 'truncated_normal/TruncatedNormal' type=TruncatedNormal>,
 <tf.Operation 'truncated_normal/mul' type=Mul>,
 <tf.Operation 'truncated_normal' type=Add>,
 <tf.Operation 'Variable' type=VariableV2>,
 <tf.Operation 'Variable/Assign' type=Assign>,
 <tf.Operation 'Variable/read' type=Identity>,
 <tf.Operation 'truncated_normal_1/shape' type=Const>,
 <tf.Operation 'truncated_normal_1/mean' type=Const>,
 <tf.Operation 'truncated_normal_1/stddev' type=Const>,
 <tf.Operation 'truncated_normal_1/TruncatedNormal' type=TruncatedNormal>,
 <tf.Operation 'truncated_normal_1/mul' type=Mul>,
 <tf.Operation 'truncated_normal_1' type=Add>,
 <tf.Operation 'Variable_1' type=VariableV2>,
 <tf.Operation 'Variable_1/Assign' type=Assign>,
 <tf.

In [18]:
# Train the model

sess = tf.Session()
sess.run(initialize_all)

num_epochs = 50
num_per_batch = 100 # divides 55000
num_iters = int(len(mnist.train.images) / num_per_batch)
for epoch in trange(num_epochs):
    for i in range(num_iters):
        start = i*num_per_batch
        end = (i+1)*num_per_batch
        sess.run(train_step, feed_dict = {X: mnist.train.images[start:end],
                                          y: mnist.train.labels[start:end]})

100%|██████████| 50/50 [01:30<00:00,  1.82s/it]


In [19]:
# See how many you get right
computed_scores = sess.run(scores, feed_dict = {X: mnist.test.images, y: mnist.test.labels})
num_test_corr = sum(np.argmax(computed_scores, axis=1) == np.argmax(mnist.test.labels, axis=1))
num_test_images = len(mnist.test.labels)
print("Got {} right out of {} images = {:2.2f}% accuracy".format(num_test_corr, num_test_images, 100.0*num_test_corr/num_test_images))

Got 9737 right out of 10000 images = 97.37% accuracy
